# Pilot 1 -- Compute the the univariate t-test contrast (whole-brain) for motor responses (LINDEX v. LMID and RINDEX v. RMID)


## Do a t-test contrast for responses within each hand separately

## Takuya Ito
#### 02/27/2018

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
import nibabel as nib
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.api as sm
import sklearn.svm as svm
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
from sklearn.feature_selection import f_classif
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [3]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 0.0 Define functions for loading data

In [5]:
def loadMotorResponses(subj):
    datadir = basedir + 'data/results/GLM_MotorResponse/' 
    data = np.loadtxt(datadir + subj + '_motorResponse_taskBetas_Surface64k_GSR.csv',delimiter=',')
    data = data[:,-4:]
    return data

# def loadRestFC(subj,fctype='pearson',gsr=True):
#     datadir = '/projects3/NetworkDiversity/data/hcppreprocessed/' + subj + '/' 
#     if fctype=='pearson':
#         if gsr:
#             data = np.loadtxt(datadir + subj + '_rest1_nuisanceResids_Glasser_GSR.csv',delimiter=',')
#         else:
#             data = np.loadtxt(datadir + subj + '_rest1_nuisanceResids_Glasser.csv',delimiter=',')
#         fc = np.corrcoef(data)
#     elif fctype=='multregfc':
#         if gsr:
#             direc = '/projects3/NetworkDiversity/data/results/multRegRestFC_GlasserParcels_HCPData/' 
#             fc = np.loadtxt(direc + subj + '_multreg_restfc_gsr.txt', delimiter=',')
#         else:
#             direc = '/projects3/NetworkDiversity/data/results/multRegRestFC_GlasserParcels_HCPData/' 
#             fc = np.loadtxt(direc + subj + '_multreg_restfc.txt', delimiter=',')
# #             fc = np.loadtxt(direc + subj + '_OutNetToFPN_multreg_restfc.txt', delimiter=',')
#     return fc
        

## 0.1 Load data

In [8]:
data_task.shape

(59412, 4, 96)

In [6]:
# gsr = True
nResponses = 4
data_task = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadMotorResponses(subj)
    scount += 1

# 1.0 Run t-test for LMID v. LINDEX and RMID v. RINDEX

In [15]:
statistics_lh = np.zeros((len(glasser2),3)) # vertices x statistics (t, p, q)
statistics_rh = np.zeros((len(glasser2),3)) # vertices x statistics (t, p, q)

t, p = stats.ttest_rel(data_task[:,0,:], data_task[:,1,:],axis=1) # lmid v. lindex
statistics_lh[:,0] = t
statistics_lh[:,1] = p
statistics_lh[:,2] = np.multiply(mc.fdrcorrection0(p)[0],t)
print 'Number of significant vertices for LMID v. LINDEX contrast:', np.sum(mc.fdrcorrection0(p)[0])

t, p = stats.ttest_rel(data_task[:,3,:], data_task[:,2,:],axis=1) # rmid v. rindex
statistics_rh[:,0] = t
statistics_rh[:,1] = p
statistics_rh[:,2] = np.multiply(mc.fdrcorrection0(p)[0],t)
print 'Number of significant vertices for RMID v. RINDEX contrast:', np.sum(mc.fdrcorrection0(p)[0])

Number of significant vertices for LMID v. LINDEX contrast: 73
Number of significant vertices for RMID v. RINDEX contrast: 206


## 1.3 Map accuracies back to cortical surface

In [16]:
  
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_MotorResponse/'
filename = 'wholeBrainUnivariateContrast_LMID_v_LINDEX'
np.savetxt(outdir + filename + '.csv', statistics_lh, fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)

outdir = '/projects3/SRActFlow/data/results/Decoding_MotorResponse/'
filename = 'wholeBrainUnivariateContrast_RMID_v_RINDEX'
np.savetxt(outdir + filename + '.csv', statistics_rh, fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0